In [ ]:
pip install qiskit

In [ ]:
pip install qiskit-aer

# Ejemplos de fase global vs fase relativa en un qubit con Qiskit.

Este script ilustra:
1. Que la fase global no altera los resultados de medición.
2. Que la fase relativa sí altera la probabilidad de obtener 0 o 1.
3. Un ejemplo con interferencia (usando fase relativa) que demuestra diferencias en la medición.


In [4]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

In [12]:
def example_1_global_phase():
    """
    Ejemplo 1:
    Comparamos |0> con e^{i*phi} |0>.
    - La fase global (phi) NO altera los resultados de medición.
    """
    print("=== Ejemplo 1: Fase Global ===")

    # Creamos dos circuitos simples
    qc1 = QuantumCircuit(1)  # Estado |0>

    qc2 = QuantumCircuit(1)  # Estado e^{i*phi}|0>
    # Asignamos una fase global al circuito (por ejemplo, pi/4)
    phi = np.pi/4
    qc2.global_phase = phi

    # Usamos Statevector para ver el estado final sin medición
    sv1 = Statevector.from_label("0").evolve(qc1)
    sv2 = Statevector.from_label("0").evolve(qc2)

    print("Estado de |0>:", sv1)
    print(f"Estado de e^(i*{phi})|0>:", sv2)
    print("Observa que la diferencia es solo una fase global.\n")

    # Ahora añadimos medición para comprobar que la distribución es la misma
    qc1_m = qc1.copy()
    qc1_m.measure_all()

    qc2_m = qc2.copy()
    qc2_m.measure_all()

    # Simulador para medir
    simulator = AerSimulator()
    shots = 1024

    qc1_m_t = transpile(qc1_m, simulator)
    qc2_m_t = transpile(qc2_m, simulator)

    result1 = simulator.run(qc1_m_t, shots=shots).result()
    result2 = simulator.run(qc2_m_t, shots=shots).result()

    counts1 = result1.get_counts(qc1_m_t)
    counts2 = result2.get_counts(qc2_m_t)

    print("Resultados de medición para |0>:", counts1)
    print(f"Resultados de medición para e^(i*{phi})|0>:", counts2)
    print("La distribución de medición es idéntica, evidenciando que la fase global no afecta.\n\n")

In [ ]:
def example_2_relative_phase():
    """
    Ejemplo 2:
    Comparamos |+> = (|0> + |1>) / sqrt(2)  con  |-> = (|0> - |1>) / sqrt(2).
    - Son estados con distinta fase relativa (±).
    - Al medir en la base Z, ambos tienen 50% de probabilidad de |0> y 50% de |1>.
    - Al medir en la base X, la diferencia se hace evidente.
    """
    print("=== Ejemplo 2: Fase Relativa (+ vs -) ===")

    # Circuito para |+>
    qc_plus = QuantumCircuit(1)
    qc_plus.h(0)  # H|0> = (|0> + |1>) / sqrt(2)

    # Circuito para |->
    qc_minus = QuantumCircuit(1)
    qc_minus.x(0)  # pasamos a |1>
    qc_minus.h(0)  # H|1> = (|0> - |1>) / sqrt(2)

    # Obtenemos los statevectors sin medición
    sv_plus = Statevector.from_label("0").evolve(qc_plus)
    sv_minus = Statevector.from_label("0").evolve(qc_minus)

    print("|+> statevector:", sv_plus)
    print("|-> statevector:", sv_minus)

    # Ahora medimos en la base Z
    qc_plus_z = qc_plus.copy()
    qc_plus_z.measure_all()
    qc_minus_z = qc_minus.copy()
    qc_minus_z.measure_all()

    simulator = AerSimulator()
    shots = 1024

    qc_plus_z_t = transpile(qc_plus_z, simulator)
    qc_minus_z_t = transpile(qc_minus_z, simulator)

    counts_plus_z = simulator.run(qc_plus_z_t, shots=shots).result().get_counts(qc_plus_z_t)
    counts_minus_z = simulator.run(qc_minus_z_t, shots=shots).result().get_counts(qc_minus_z_t)

    print("\nMedición en la base Z (estados |0>, |1>):")
    print("Counts |+>:", counts_plus_z)
    print("Counts |->:", counts_minus_z)

    # Medición en la base X -> aplicar H antes de medir en Z
    qc_plus_x = qc_plus.copy()
    qc_plus_x.h(0)
    qc_plus_x.measure_all()

    qc_minus_x = qc_minus.copy()
    qc_minus_x.h(0)
    qc_minus_x.measure_all()

    qc_plus_x_t = transpile(qc_plus_x, simulator)
    qc_minus_x_t = transpile(qc_minus_x, simulator)

    counts_plus_x = simulator.run(qc_plus_x_t, shots=shots).result().get_counts(qc_plus_x_t)
    counts_minus_x = simulator.run(qc_minus_x_t, shots=shots).result().get_counts(qc_minus_x_t)

    print("\nMedición en la base X (estados |+>, |->):")
    print("Counts |+> (en base X):", counts_plus_x)
    print("Counts |-> (en base X):", counts_minus_x)

    print("\nLa fase relativa (+ vs -) no influye en la medición Z (ambos 50-50), "
          "pero sí cambia la medición en la base X.\n\n")

In [ ]:
def example_3_interference():
    """
    Ejemplo 3:
    Ilustramos cómo la fase relativa genera interferencia.
    - Preparamos un qubit en (|0> + e^{i*phi}|1>) / sqrt(2).
    - Luego aplicamos otra H y medimos.
    - Observamos cómo cambian los resultados según la fase relativa.
    """
    print("=== Ejemplo 3: Interferencia por Fase Relativa ===")

    simulator = AerSimulator()
    shots = 1024

    for phi in [0, np.pi/2, np.pi, 3*np.pi/2]:
        qc = QuantumCircuit(1)
        # 1) Empezamos en |0>
        # 2) Aplicamos H -> (|0> + |1>)/sqrt(2)
        qc.h(0)

        # 3) Agregamos fase 'phi' solo a la componente |1>
        qc.p(phi, 0)

        # 4) Aplicamos H de nuevo para ver interferencia
        qc.h(0)

        # 5) Medimos
        qc.measure_all()

        # Transpilamos y ejecutamos
        qc_t = transpile(qc, simulator)
        result = simulator.run(qc_t, shots=shots).result()
        counts = result.get_counts(qc_t)

        print(f"Fase relativa phi = {phi} rad ({phi*180/np.pi:.1f}°): {counts}")

    print("\nLos distintos valores de phi cambian la interferencia y, por lo tanto, "
          "las probabilidades finales de medir |0> o |1>.\n")

In [13]:
example_1_global_phase()

=== Ejemplo 1: Fase Global ===
Estado de |0>: Statevector([1.+0.j, 0.+0.j],
            dims=(2,))
Estado de e^(i*0.7853981633974483)|0>: Statevector([0.70710678+0.70710678j, 0.        +0.j        ],
            dims=(2,))
Observa que la diferencia es solo una fase global.

Resultados de medición para |0>: {'0': 1024}
Resultados de medición para e^(i*0.7853981633974483)|0>: {'0': 1024}
La distribución de medición es idéntica, evidenciando que la fase global no afecta.




In [ ]:
example_2_relative_phase()

=== Ejemplo 2: Fase Relativa (+ vs -) ===
|+> statevector: Statevector([0.70710678+0.j, 0.70710678+0.j],
            dims=(2,))
|-> statevector: Statevector([ 0.70710678+0.j, -0.70710678+0.j],
            dims=(2,))

Medición en la base Z (estados |0>, |1>):
Counts |+>: {'0': 497, '1': 527}
Counts |->: {'0': 490, '1': 534}

Medición en la base X (estados |+>, |->):
Counts |+> (en base X): {'0': 1024}
Counts |-> (en base X): {'1': 1024}

La fase relativa (+ vs -) no influye en la medición Z (ambos 50-50), pero sí cambia la medición en la base X.




In [ ]:
example_3_interference()

=== Ejemplo 3: Interferencia por Fase Relativa ===
Fase relativa phi = 0 rad (0.0°): {'0': 1024}
Fase relativa phi = 1.5707963267948966 rad (90.0°): {'1': 492, '0': 532}
Fase relativa phi = 3.141592653589793 rad (180.0°): {'1': 1024}
Fase relativa phi = 4.71238898038469 rad (270.0°): {'0': 503, '1': 521}

Los distintos valores de phi cambian la interferencia y, por lo tanto, las probabilidades finales de medir |0> o |1>.

